# Reactor agua: Acuña, Guadalupe y Molina, Nicolás

In [217]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time
import scipy.signal
from scipy.optimize import curve_fit
import scipy.stats as stats
import pandas as pd
import os
from  scipy.stats import chi2_contingency
from statsmodels.stats.weightstats import DescrStatsW
import glob
from scipy.signal import butter, filtfilt

#os.chdir (r'C:\Users\Sergio\Desktop\labo5\difractiva\codigos')
#os.chdir (r'C:\Users\Nicolás Molina\Desktop\L5\difractiva\codigos')

#get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('matplotlib', 'qt5')

def periodo(t_señal, señal, percent_tol = 10, separacion_max = 100):
    inds_max = [i for i in range(len(señal))
        if señal[i] >= max(señal)*(1-percent_tol/100)]

    ind_periodos = [0] + [i+1 for i in range(len(inds_max) -1)
        if (inds_max[i+1]-inds_max[i])>separacion_max ] + [len (inds_max)]

    ventanas = [ inds_max[ind_periodos [i]:ind_periodos [i+1]]
        for i in range(len(ind_periodos) -1)]

    picos_index = [round(np.mean(i)) for i in ventanas]

    tper = np.mean([(t_señal[picos_index[i+1]]-t_señal[picos_index[i]]) for i in range(len(picos_index) -1)])
                   
    iper = round(np.mean([(picos_index[i+1]-picos_index[i]) for i in range (len(picos_index) -1)]))

    return tper, iper

## limpio txts ARREGLAR

In [218]:
import pandas as pd
import glob
os.chdir (r'C:\Users\Nicolás Molina\Desktop\L6-7\prueba')

# Obtener la lista de archivos CSV en el directorio actual
csv_files = glob.glob('*.csv')

# Loop a través de cada archivo CSV
for file in csv_files:
    # Cargar el archivo CSV en un dataframe de pandas
    df =pd.read_csv(file)
    
    # Reemplazar todas las ocurrencias de ",," por ",0," en el dataframe
    df.replace(',,',',0,', regex=True, inplace=True)
    
    # Guardar el dataframe modificado en el mismo archivo CSV
    df.to_csv(file, index=False)


In [219]:
import pandas as pd
import os

# Definir el directorio que contiene los archivos de texto
directory = '/Users/Nicolás Molina/Desktop/L6-7/prueba'


# Obtener la lista de archivos CSV en el directorio
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Iterar sobre cada archivo CSV y reemplazar el valor
for file in csv_files:
    # Leer el archivo CSV en un DataFrame
    df = pd.read_csv(os.path.join(directory, file))

    # Reemplazar los valores utilizando el método replace()
    df = df.replace(',,,', ',0,0,', regex=True)

    # Guardar el DataFrame modificado como un archivo CSV
    df.to_csv(os.path.join(directory, file), index=False)



## datos crudos todos juntos (para elegir cual ver)

In [220]:
os.chdir (r'C:\Users\Nicolás Molina\Desktop\L6-7\13-4-23')
files=glob.glob('*.csv')

plt.close("all")

for file in files:
    med = np.loadtxt(file, delimiter=',', skiprows=16, unpack=True)
    t=med[3]
    Vdbd=med[4]
    Vstr=med[8]
    Istr=Vstr/50 *1000


    fig, (ax1, ax2) = plt.subplots(2, 1)


    # Graficar los datos en el primer subplot
    ax1.plot(t, Vdbd, color='blue', label='Vdbd')
    ax1.grid()
    ax1.set_title("Voltaje de alimentación")
    ax1.set_xlabel('tiempos (s)')
    ax1.set_ylabel('V')
    ax1.legend()

    # Graficar los datos en el segundo subplot
    ax2.plot(t, Istr, color='red', label='Istr')
    ax2.grid()
    ax2.set_title("Corriente de streamers")
    ax2.set_xlabel('tiempo (s)')
    ax2.set_ylabel('I [mA]')
    ax2.legend()

    fig.suptitle("archivo="+file)

    plt.show()

## datos crudos (un solo archivo)

In [221]:
#file="1ppm_C 2023-04-13 14h 56m 08s.csv"
file="1ppm_C 2023-04-13 14h 55m 55s.csv"
plt.close("all")

#os.chdir (r'C:\Users\Sergio\Desktop\labo5\difractiva\codigos\espectroscopia')
os.chdir (r'C:\Users\Nicolás Molina\Desktop\L6-7\13-4-23')


med = np.loadtxt(file, delimiter=',', skiprows=16, unpack=True)
t=med[3]
Vdbd=med[4]
Vstr=med[8]
Istr=Vstr/50 *1000
"""
plt.figure()
plt.grid()
plt.title("Voltaje de alimentación")
plt.plot(t,Vdbd)
plt.xlabel("tiempo (s)")
plt.ylabel("V")
#plt.legend()

plt.figure()
plt.grid()
plt.title("Corriente de streamers")
plt.plot(t,Istr)
plt.xlabel("tiempo (s)")
plt.ylabel("I (mA)")
"""

fig, (ax1, ax2) = plt.subplots(2, 1)


# Graficar los datos en el primer subplot
ax1.plot(t, Vdbd, color='blue', label='Vdbd')
ax1.grid()
ax1.set_title("Voltaje de alimentación")
ax1.set_xlabel('tiempos (s)')
ax1.set_ylabel('V')
ax1.legend()

# Graficar los datos en el segundo subplot
ax2.plot(t, Istr, color='red', label='Istr')
ax2.grid()
ax2.set_title("Corriente de streamers")
ax2.set_xlabel('tiempo (s)')
ax2.set_ylabel('I [mA]')
ax2.legend()

plt.show()




## limpio los datos

In [222]:
def sin(x,T,a,b):
    y=a*np.sin(2*np.pi/T*x)+b
    return y

init_vals=[1/8000,2.5,0]

plt.close("all")

popt, pcov = curve_fit(sin, t, Istr,absolute_sigma=True,p0=init_vals)       #,p0=init_vals
perr = np.sqrt(np.diag(pcov))

valorT= popt[0]
valora=popt[1]
valorb=popt[2]
err_T = perr[0]
print("periodo=",valorT)
print("frecuencia=",1/valorT)
print("len",len(t))
print("tiempo capturado en pantalla=",t[-1]-t[0])
longper=valorT*10000000
longper=round(longper)
print(longper)

#print("valores inversa",valora1,valorb1)
bins=np.linspace(t[0],t[-1],len(t))

ajuste= sin(bins,valorT,valora,valorb)

plt.figure()
plt.grid()
plt.title("Corriente de streamers")
plt.plot(t,Istr,label="datos crudos")
plt.plot(bins,ajuste,label="ajuste seno")
plt.xlabel("tiempo (s)")
plt.ylabel("I (mA)")
plt.legend()

frecuencia=1/valorT


signal=Istr

# Definir la frecuencia de corte del filtro
cutoff = 100

# Crear el filtro Butterworth de orden elevado
order = 5
nyquist = 0.5 * frecuencia
normal_cutoff = cutoff / nyquist
b, a = butter(order, normal_cutoff, btype='high')

# Aplicar el filtro a la señal
filtered_signal = filtfilt(b, a, signal)

Istr_plana=filtered_signal

# Graficar la señal original y la señal filtrada
plt.figure()
plt.grid()
plt.title("Aplanamiento de señal")
plt.plot(t, signal, label='Señal original')
plt.plot(t, filtered_signal, label='Señal filtrada')
plt.xlabel("tiempo (s)")
plt.ylabel("I [mA]")
plt.legend()
plt.show()

periodo= 0.00013009911236109523
frecuencia= 7686.447523365574
len 2489
tiempo capturado en pantalla= 0.0002488
1301


## aplanamiento final de la señal convirtiendo el ruido en 0

In [223]:

p = 1 #valor umbral

# Recorrer la lista y actualizar los valores menores a 'p' a cero
for i in range(len(Istr_plana)):
    if Istr_plana[i] < p:
        Istr_plana[i] = 0
        

plt.figure()
plt.grid()
plt.title("Aplanamiento total de señal")
plt.plot(t, Istr_plana, label='Señal filtrada')
plt.xlabel("tiempo (s)")
plt.ylabel("I [mA]")
plt.legend()
plt.show()

## FINAL

In [227]:
plt.close("all")

#os.chdir (r'C:\Users\Sergio\Desktop\labo5\difractiva\codigos\espectroscopia')
os.chdir (r'C:\Users\Nicolás Molina\Desktop\L6-7\13-4-23')
#el len de los archivos es 2489
# [500:500+longper]
med = np.loadtxt(file, delimiter=',', skiprows=16, unpack=True)
t=med[3]
Vdbd=med[4]
Vstr=med[8]
Istr=Vstr/50 *1000
"""
plt.figure()
plt.grid()
plt.title("Voltaje de alimentación")
plt.plot(t,Vdbd)
plt.xlabel("tiempo (s)")
plt.ylabel("V")
#plt.legend()

plt.figure()
plt.grid()
plt.title("Corriente de streamers")
plt.plot(t,Istr)
plt.xlabel("tiempo (s)")
plt.ylabel("I (mA)")
"""

fig, (ax1, ax2) = plt.subplots(2, 1)


# Graficar los datos en el primer subplot
ax1.plot(t[1000:1000+longper], Vdbd[1000:1000+longper], color='blue', label='Vdbd')
ax1.grid()
ax1.set_title("Voltaje de alimentación")
ax1.set_xlabel('tiempos (s)')
ax1.set_ylabel('V')
ax1.legend()

# Graficar los datos en el segundo subplot
ax2.plot(t[1000:1000+longper], Istr_plana[1000:1000+longper], color='red', label='Istr')
ax2.grid()
ax2.set_title("Corriente de streamers")
ax2.set_xlabel('tiempo (s)')
ax2.set_ylabel('I [mA]')
ax2.legend()


plt.show()
Vpot=Vdbd[1000:1000+longper]
Ipot=Istr[1000:1000+longper]/1000 #paso la corriente a A
N=len(Ipot)
potencia=np.mean(Vpot*Ipot/N)
print("potencia=",potencia, "W")

potencia= 0.005898188645758805 W


In [225]:
"""
periodo(t,Vdbd)

print(1/tper)

periodo(t,Istr)

print(1/tper)
"""

'\nperiodo(t,Vdbd)\n\nprint(1/tper)\n\nperiodo(t,Istr)\n\nprint(1/tper)\n'